In [2]:
from collections import Counter
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np
from scipy.stats.stats import pearsonr

In [3]:
already_extracted=True
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

if already_extracted:
    df = pd.read_pickle('../reviews_Kindle_Store_5.h5')
else:
    df = getDF('../reviews_Kindle_Store_5.json.gz')


In [4]:
df.shape

(982619, 11)

In [5]:
from sklearn.utils import shuffle
#df.to_csv('reviews_Books_5.csv')
df = shuffle(df)

In [6]:
from sklearn.model_selection import train_test_split

Training_DataFrame, Testing_DataFrame = train_test_split(df, test_size=0.2)

In [7]:
print df[:2]

            reviewerID        asin reviewerName helpful  unixReviewTime  \
153879  A1PPWRKOYJ6CO5  B006NY4UPS   Vera Green  [0, 0]      1370736000   
208779  A2C9QDWHXQIHY0  B007SPPH1M        J3818  [1, 1]      1385856000   

                                               reviewText  overall  \
153879  Loved the smell and feel of old London.  Loved...      5.0   
208779  This book was ok. I didn't love it, I thought ...      2.0   

        reviewTime     summary  sentiments  corrected_sent_score  
153879  06 9, 2013  Great read       0.836                 0.336  
208779  12 1, 2013          Ok       0.481                 0.000  


In [8]:
print Training_DataFrame.shape
print Testing_DataFrame.shape

(786095, 11)
(196524, 11)


In [9]:
print "Training dataset details:"
Uniq_items_Train =  len(set(Training_DataFrame['asin'].values))
print "Number of item:", Uniq_items_Train
Uniq_users_Train =   len(set(Training_DataFrame['reviewerID'].values))
print "Number of Users:", Uniq_users_Train
Uniq_ratings_Train = len(set(Training_DataFrame['overall'].values))
print "Number of ratings:", Uniq_ratings_Train
print "\n"

Training dataset details:
Number of item: 61929
Number of Users: 68218
Number of ratings: 5




In [10]:
#Training_DataFrame=Training_DataFrame[:5000]
Training_DataFrame.columns
# Lists of unique items and users
item_list = list(set(Training_DataFrame['asin'].values))
user_list = list(set(Training_DataFrame['reviewerID'].values))

#Storing the indices of users to generate the Rating Memory Matrix
UserIndices={}
for i in range(len(user_list)):
    UserIndices[user_list[i]] = i

#Storing the indices of items to generate the Rating Memory Matrix
#These indices will be used to populate the UsersRatingMemory for Memory based Collaborative Filtering
itemIndices={}
for j in range(len(item_list)):
    itemIndices[item_list[j]] = j
    

# Collaborative Filtering without Sentiment Scores: 

In [9]:
#UsersRatingMemory stores the users and corresponding items ratings in Matrix form
UsersRatingMemory=np.zeros((Uniq_users_Train,Uniq_items_Train))

for i in range(0, len(Training_DataFrame)):
    item_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and item indices
    user_ind=UserIndices[user_id]
    item_ind=itemIndices[item_id]
    rating=Training_DataFrame.iloc[i]['overall']
    #populate the matrix with rating at corresponding user and item index
    UsersRatingMemory[user_ind][item_ind]=float(rating)

AvgRatings = np.true_divide(UsersRatingMemory.sum(1),(UsersRatingMemory!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRatings

Average Ratings of the Users are:
[4.96666667 5.         5.         ... 3.16666667 3.42857143 4.67307692]


In [10]:
from scipy.stats.stats import pearsonr

def PredictUserRating(user_id, item_id):
    '''This function predicts the rating based on user_id and item_id'''
    item_ind = itemIndices[item_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the item
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory[:,item_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #itemRatingsOthers: stores other users ratings
    itemRatingsOthers = UsersRatingMemory[ratedindices, item_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory[ind],UsersRatingMemory[user_ind])[0])
    k=0.001
    # Calculate the prediction
    pred = AvgRatings[user_ind] + k * np.sum( PearsonCoeffs* (itemRatingsOthers - AvgRatings[ratedindices]))
    return pred

def Test(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        item_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRating(user_id,item_id)
    return Predictions

In [41]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
Predictions=Test(1000)
print "For 1000 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'].values[:1000])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:1000]))

Index([u'reviewerID', u'asin', u'reviewerName', u'helpful', u'unixReviewTime',
       u'reviewText', u'overall', u'reviewTime', u'summary'],
      dtype='object')
For 10 Samples:
The Mean Absolute Error is: 0.5449603142830374
Root Mean Square Error is: 0.8166608941062132


In [11]:
def get_other_closest_user(user_ind,k):
    PearsonCoeffs={}
    for ind in range(len(user_list)):
        if ind==user_ind :
            continue
        else:
            PearsonCoeffs[ind]=pearsonr(UsersRatingMemory[ind],UsersRatingMemory[user_ind])[0]
        
    PearsonCoeffs_sorted=sorted(PearsonCoeffs.items(), key=lambda kv: kv[1], reverse=True)
    return [x for x, y in PearsonCoeffs_sorted[:k]]

In [61]:
colsest_users=get_other_closest_user(0,20)
#colsest_users_sorted=sorted(colsest_users.items(), key=lambda kv: kv[1], reverse=True)
#colsest_users=[x for x, y in colsest_users_sorted[:20]]
print colsest_users

KeyboardInterrupt: 

In [12]:
def recommender(user_ind):
    items_to_recommend=[]
    colsest_users=get_other_closest_user(user_ind,20)
    for user in colsest_users:
        for items in np.nonzero(UsersRatingMemory[user]):
            for item in items:
                if UsersRatingMemory[user_ind][item] == 0:
                    items_to_recommend.append(item_list[item])
    
    recommendations = {}
    for item in items_to_recommend:
        recommendations[item] = PredictUserRating(user_list[user_ind], item)
        
    sorted_recommendations = sorted(recommendations.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_recommendations

In [13]:
print "getting recommendations for user:", user_list[0]
print "Top 50 recommendations are For Collaborative Filtering without sentiment Scores ..."
recommendations=recommender(0)
print recommendations[:50]

getting recommendations for user: AO00I2TVC9LBB
Top 10 recommendations are For Collaborative Filtering without sentiment Scores ...
[('B00KIXIBYQ', 4.966666666666667), ('B00KBDWHKM', 4.966666666666667), ('B00BTPOFSI', 4.966666666666667), ('B00FR3ZCG2', 4.966666666666667), ('B00GW7NHXM', 4.966666666666667), ('B00IW3BOC0', 4.966666666666667), ('B00KKQACDO', 4.966666666666667), ('B00AMIH8Y6', 4.966666666666667), ('B00L2FQEVG', 4.966666666666667), ('B00CMXCWW2', 4.966666666666667)]


# Colloboarative Filtering with Sentiment Scores:

In [12]:
UsersRatingMemory_sentiment=np.zeros((Uniq_users_Train,Uniq_items_Train))

for i in range(0, len(Training_DataFrame)):
    item_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and item indices
    user_ind=UserIndices[user_id]
    item_ind=itemIndices[item_id]
    rating=float(Training_DataFrame.iloc[i]['overall'])
    
    if Training_DataFrame.iloc[i]['reviewText']:
    #populate the matrix with rating at corresponding user and item index
        rating=rating + float(Training_DataFrame.iloc[i]['corrected_sent_score'])
    
    UsersRatingMemory_sentiment[user_ind][item_ind]=rating

AvgRating_sentiment = np.true_divide(UsersRatingMemory_sentiment.sum(1),(UsersRatingMemory_sentiment!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRating_sentiment

Average Ratings of the Users are:
[5.29856    5.24056552 4.8829     ... 3.79838    3.97384286 4.69051373]


In [22]:

def PredictUserRatingsentiment(user_id, item_id):
    '''This function predicts the rating based on user_id and item_id'''
    item_ind = itemIndices[item_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the item
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory_sentiment[:,item_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #itemRatingsOthers: stores other users ratings
    itemRatingsOthers = UsersRatingMemory_sentiment[ratedindices, item_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory_sentiment[ind],UsersRatingMemory_sentiment[user_ind])[0])
    k=0.001
    # Calculate the prediction
    pred = AvgRating_sentiment[user_ind] + k * np.sum( PearsonCoeffs* (itemRatingsOthers - AvgRating_sentiment[ratedindices]))
    return pred

def Testsentiment(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        item_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRatingsentiment(user_id,item_id)
    return Predictions

In [23]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
Predictions=Testsentiment(1000)
print "For 1000 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'].values[:1000])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:1000]))

KeyboardInterrupt: 

In [24]:
def get_other_closest_user_sentiment(user_ind,k):
    PearsonCoeffs={}
    for ind in range(len(user_list)):
        if ind==user_ind :
            continue
        else:
            PearsonCoeffs[ind]=pearsonr(UsersRatingMemory_sentiment[ind],UsersRatingMemory_sentiment[user_ind])[0]
        
    PearsonCoeffs_sorted=sorted(PearsonCoeffs.items(), key=lambda kv: kv[1], reverse=True)
    return [x for x, y in PearsonCoeffs_sorted[:k]]

In [25]:
def recommender_sentiments(user_ind):
    items_to_recommend=[]
    colsest_users=get_other_closest_user_sentiment(user_ind,20)
    for user in colsest_users:
        for items in np.nonzero(UsersRatingMemory_sentiment[user]):
            for item in items:
                if UsersRatingMemory_sentiment[user_ind][item] == 0:
                    items_to_recommend.append(item_list[item])
    
    recommendations = {}
    for item in items_to_recommend:
        recommendations[item] = PredictUserRatingsentiment(user_list[user_ind], item)
        
    sorted_recommendations = sorted(recommendations.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_recommendations

In [26]:
print "getting recommendations for user:", user_list[1]
print "Top 50 recommendations are For Collaborative Filtering with sentiment Scores ..."
recommendations_sentiment=recommender_sentiments(1)


getting recommendations for user: AO00I2TVC9LBB
Top 50 recommendations are For Collaborative Filtering with sentiment Scores ...
[('B00KIXIBYQ', 5.240565517241379), ('B00KBDWHKM', 5.240565517241379), ('B00BTPOFSI', 5.240565517241379), ('B00FR3ZCG2', 5.240565517241379), ('B00GW7NHXM', 5.240565517241379), ('B00IW3BOC0', 5.240565517241379), ('B00BVR73P6', 5.240565517241379), ('B00AMIH8Y6', 5.240565517241379), ('B00CMXCWW2', 5.240565517241379), ('B00GM4Y0OU', 5.240565517241379), ('B00KLPFCJI', 5.240565517241379), ('B00FK6WG6K', 5.240565517241379), ('B00HHEVSK8', 5.240565517241379), ('B0058PIWXY', 5.240565517241379), ('B0089MXR6K', 5.240565517241379), ('B004PLNH5K', 5.240565517241379), ('B0069F1LLM', 5.240565517241379), ('B00K0YBVIG', 5.240565517241379), ('B00KXIIZKG', 5.240565517241379), ('B00IZCODEE', 5.240565517241379), ('B008EKOPGS', 5.240565517241379), ('B00F7CJKAC', 5.240565517241379), ('B008OPADK0', 5.240565517241379), ('B00CYBFTWC', 5.240565517241379), ('B00BVA2QBO', 5.2405655172413

1
